# Plagiarism Detection Model

Now that you've created training and test data, you are ready to define and train a model. Your goal in this notebook, will be to train a binary classification model that learns to label an answer file as either plagiarized or not, based on the features you provide the model.

This task will be broken down into a few discrete steps:

* Upload your data to S3.
* Define a binary classification model and a training script.
* Train your model and deploy it.
* Evaluate your deployed classifier and answer some questions about your approach.

To complete this notebook, you'll have to complete all given exercises and answer all the questions in this notebook.
> All your tasks will be clearly labeled **EXERCISE** and questions as **QUESTION**.

It will be up to you to explore different classification models and decide on a model that gives you the best performance for this dataset.

---

## Load Data to S3

In the last notebook, you should have created two files: a `training.csv` and `test.csv` file with the features and class labels for the given corpus of plagiarized/non-plagiarized text data. 

>The below cells load in some AWS SageMaker libraries and creates a default bucket. After creating this bucket, you can upload your locally stored data to S3.

Save your train and test `.csv` feature files, locally. To do this you can run the second notebook "2_Plagiarism_Feature_Engineering" in SageMaker or you can manually upload your files to this notebook using the upload icon in Jupyter Lab. Then you can upload local files to S3 by using `sagemaker_session.upload_data` and pointing directly to where the training data is saved.

In [1]:
import pandas as pd
import numpy as np
import boto3
import sagemaker
import os

In [2]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# create an S3 bucket
bucket = sagemaker_session.default_bucket()

## EXERCISE: Upload your training data to S3

Specify the `data_dir` where you've saved your `train.csv` file. Decide on a descriptive `prefix` that defines where your data will be uploaded in the default S3 bucket. Finally, create a pointer to your training data by calling `sagemaker_session.upload_data` and passing in the required parameters. It may help to look at the [Session documentation](https://sagemaker.readthedocs.io/en/stable/session.html#sagemaker.session.Session.upload_data) or previous SageMaker code examples.

You are expected to upload your entire directory. Later, the training script will only access the `train.csv` file.

In [3]:
# should be the name of directory you created to save your features data
data_dir = 'plagiarism_data'

# set prefix, a descriptive name for a directory  
prefix = 'plagiarism_detection'

# upload all data to S3
train_and_test_path = sagemaker_session.upload_data(path=data_dir, bucket=bucket, key_prefix=prefix)

In [4]:
print(train_and_test_path)

s3://sagemaker-eu-central-1-426020052183/plagiarism_detection


### Test cell

Test that your data has been successfully uploaded. The below cell prints out the items in your S3 bucket and will throw an error if it is empty. You should see the contents of your `data_dir` and perhaps some checkpoints. If you see any other files listed, then you may have some old model files that you can delete via the S3 console (though, additional files shouldn't affect the performance of model developed in this notebook).

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# confirm that data is in S3 bucket
empty_check = []
for obj in boto3.resource('s3').Bucket(bucket).objects.all():
    empty_check.append(obj.key)
    print(obj.key)

assert len(empty_check) !=0, 'S3 bucket is empty.'
print('Test passed!')

plagiarism_detection/test.csv
plagiarism_detection/train.csv
plagiarism_output/linear-learner-2020-12-13-23-45-21-363/output/model.tar.gz
plagiarism_output/linear-learner-2020-12-13-23-45-21-363/profiler-output/framework/training_job_end.ts
plagiarism_output/linear-learner-2020-12-13-23-45-21-363/profiler-output/system/incremental/2020121323/1607903220.algo-1.json
plagiarism_output/linear-learner-2020-12-13-23-45-21-363/profiler-output/system/incremental/2020121323/1607903280.algo-1.json
plagiarism_output/linear-learner-2020-12-13-23-45-21-363/profiler-output/system/training_job_end.ts
plagiarism_output/linear-learner-2020-12-13-23-45-21-363/rule-output/ProfilerReport-1607903121/profiler-output/profiler-report.html
plagiarism_output/linear-learner-2020-12-13-23-45-21-363/rule-output/ProfilerReport-1607903121/profiler-output/profiler-report.ipynb
plagiarism_output/linear-learner-2020-12-13-23-45-21-363/rule-output/ProfilerReport-1607903121/profiler-output/profiler-reports/BatchSize.json

---

# Modeling

Now that you've uploaded your training data, it's time to define and train a model!

The type of model you create is up to you. For a binary classification task, you can choose to go one of three routes:
* Use a built-in classification algorithm, like LinearLearner.
* Define a custom Scikit-learn classifier, a comparison of models can be found [here](https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html).
* Define a custom PyTorch neural network classifier. 

It will be up to you to test out a variety of models and choose the best one. Your project will be graded on the accuracy of your final model. 
 
---

## EXERCISE: Complete a training script 

To implement a custom classifier, you'll need to complete a `train.py` script. You've been given the folders `source_sklearn` and `source_pytorch` which hold starting code for a custom Scikit-learn model and a PyTorch model, respectively. Each directory has a `train.py` training script. To complete this project **you only need to complete one of these scripts**; the script that is responsible for training your final model.

A typical training script:
* Loads training data from a specified directory
* Parses any training & model hyperparameters (ex. nodes in a neural network, training epochs, etc.)
* Instantiates a model of your design, with any specified hyperparams
* Trains that model 
* Finally, saves the model so that it can be hosted/deployed, later

### Defining and training a model
Much of the training script code is provided for you. Almost all of your work will be done in the `if __name__ == '__main__':` section. To complete a `train.py` file, you will:
1. Import any extra libraries you need
2. Define any additional model training hyperparameters using `parser.add_argument`
2. Define a model in the `if __name__ == '__main__':` section
3. Train the model in that same section

Below, you can use `!pygmentize` to display an existing `train.py` file. Read through the code; all of your tasks are marked with `TODO` comments. 

**Note: If you choose to create a custom PyTorch model, you will be responsible for defining the model in the `model.py` file,** and a `predict.py` file is provided. If you choose to use Scikit-learn, you only need a `train.py` file; you may import a classifier from the `sklearn` library.

In [6]:
# directory can be changed to: source_sklearn or source_pytorch
!pygmentize source_sklearn/train.py

from __future__ import print_function

import argparse
import os
import pandas as pd
import numpy as np

# sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. 
# from sklearn.externals import joblib
# Import joblib package directly
import joblib

## TODO: Import any additional libraries you need to define a model
### Solution 2: Adaboost
#from sklearn.ensemble import AdaBoostClassifier
### Solution 3: Neural Net MLPClassifier
from sklearn.neural_network import MLPClassifier
### Solution 4: Nearest Neighbors
#from sklearn.neighbors import KNeighborsClassifier
### Solution 5: Decision Tree
#from sklearn.tree import DecisionTreeClassifier

# Provided model load function
def model_fn(model_dir):
    """Load model from the model_dir. This is the same model that is saved
    in the main if statement.
    """
    print("Loading model.")
    
    # load using joblib
    # joblib replaced by numpy.load due to error on executing train.py, see https://knowledge.udacity.com

### Provided code

If you read the code above, you can see that the starter code includes a few things:
* Model loading (`model_fn`) and saving code
* Getting SageMaker's default hyperparameters
* Loading the training data by name, `train.csv` and extracting the features and labels, `train_x`, and `train_y`

If you'd like to read more about model saving with [joblib for sklearn](https://scikit-learn.org/stable/modules/model_persistence.html) or with [torch.save](https://pytorch.org/tutorials/beginner/saving_loading_models.html), click on the provided links.

---
# Create an Estimator

When a custom model is constructed in SageMaker, an entry point must be specified. This is the Python file which will be executed when the model is trained; the `train.py` function you specified above. To run a custom training script in SageMaker, construct an estimator, and fill in the appropriate constructor arguments:

* **entry_point**: The path to the Python script SageMaker runs for training and prediction.
* **source_dir**: The path to the training script directory `source_sklearn` OR `source_pytorch`.
* **entry_point**: The path to the Python script SageMaker runs for training and prediction.
* **source_dir**: The path to the training script directory `train_sklearn` OR `train_pytorch`.
* **entry_point**: The path to the Python script SageMaker runs for training.
* **source_dir**: The path to the training script directory `train_sklearn` OR `train_pytorch`.
* **role**: Role ARN, which was specified, above.
* **train_instance_count**: The number of training instances (should be left at 1).
* **train_instance_type**: The type of SageMaker instance for training. Note: Because Scikit-learn does not natively support GPU training, Sagemaker Scikit-learn does not currently support training on GPU instance types.
* **sagemaker_session**: The session used to train on Sagemaker.
* **hyperparameters** (optional): A dictionary `{'name':value, ..}` passed to the train function as hyperparameters.

Note: For a PyTorch model, there is another optional argument **framework_version**, which you can set to the latest version of PyTorch, `1.0`.

## EXERCISE: Define a Scikit-learn or PyTorch estimator

To import your desired estimator, use one of the following lines:
```
from sagemaker.sklearn.estimator import SKLearn
```
```
from sagemaker.pytorch import PyTorch
```

In [38]:

# your import and estimator code, here

# specify an output path
prefix = 'plagiarism_output'
output_path = 's3://{}/{}'.format(bucket, prefix)

# specify the source dir (for SKLearn and PyTorch only)
source_dir = 'source_sklearn'
#source_dir = source_pytorch

### Solution 1: AWS LinearLearner
from sagemaker import LinearLearner
#
# instantiate LinearLearner
linear = LinearLearner(role=role,
                       train_instance_count=1, 
                       train_instance_type='ml.c4.xlarge',
                       predictor_type='binary_classifier',
                       output_path=output_path,
                       sagemaker_session=sagemaker_session,
                       epochs=15,
                       binary_classifier_model_selection_criteria='precision_at_target_recall', 
                       target_precision=0.9,
                       positive_example_weight_mult='balanced'
                      )
### End Solution 1

### Solution 2: sklearn Adaboost
#from sagemaker.sklearn.estimator import SKLearn
#
#estimator = SKLearn(entry_point='train.py',
#                    source_dir=source_dir,
#                    role=role,
#                    train_instance_count=1, 
#                    train_instance_type= 'ml.c4.xlarge', #'ml.m4.xlarge'
#                    output_path=output_path,
#                    sagemaker_session=sagemaker_session,
#                    framework_version='0.23-1',
#                    py_version='py3',                    
#                    hyperparameters = {'n_estimators': 60, 'learning_rate': 1.2}
#                   )
### End Solution 2

### Solution 3: Neural Net MLPClassifier
#from sagemaker.sklearn.estimator import SKLearn
#
#estimator = SKLearn(entry_point='train.py',
#                    source_dir=source_dir,
#                    role=role,
#                    train_instance_count=1, 
#                    train_instance_type= 'ml.c4.xlarge', #'ml.m4.xlarge'
#                    output_path=output_path,                
#                    sagemaker_session=sagemaker_session,
#                    #image_name=image_name,
#                    framework_version='0.23-1',
#                    py_version='py3',
#                    hyperparameters = {'hidden_layer_sizes': '60 100 60', 
#                                       'activation': 'relu',
#                                       'solver': 'lbfgs', # instead of adam, use lbfgs for small dataset
#                                       'learning_rate_init': 0.005
#                                      }
#                   )
### End Solution 3

### Solution 4: Nearest Neighbors
#from sagemaker.sklearn.estimator import SKLearn
#
#estimator = SKLearn(entry_point='train.py',
#                    source_dir=source_dir,
#                    role=role,
#                    train_instance_count=1, 
#                    train_instance_type= 'ml.c4.xlarge', #'ml.m4.xlarge'
#                    output_path=output_path,
#                    sagemaker_session=sagemaker_session,
#                    #image_name=image_name,
#                    framework_version='0.23-1',
#                    py_version='py3',
#                    hyperparameters = {'n_neighbors': 3, 
#                                       'weights': 'uniform',
#                                       'algorithm': 'auto',
#                                       'leaf_size': 30
#                                      }
#                   )
### End Solution 4

### Solution 5: Decision Tree
#from sagemaker.sklearn.estimator import SKLearn
#
#estimator = SKLearn(entry_point='train.py',
#                    source_dir=source_dir,
#                    role=role,
#                    train_instance_count=1, 
#                    train_instance_type= 'ml.c4.xlarge', #'ml.m4.xlarge'
#                    output_path=output_path,
#                    sagemaker_session=sagemaker_session,
#                    #image_name=image_name,
#                    framework_version='0.23-1',
#                    py_version='py3',
#                    hyperparameters = {'criterion': 'gini', 
#                                       'splitter': 'best',
#                                       'max_depth': 5
#                                      }
#                   )
### End Solution 5

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


## EXERCISE: Train the estimator

Train your estimator on the training data stored in S3. This should create a training job that you can monitor in your SageMaker console.

In [39]:
%%time

# Train your estimator on S3 training data

### Solution 1: LinearLearner
train_data = pd.read_csv(os.path.join(train_and_test_path, "train.csv"), header=None, names=None)
#
# Labels are in the first column
train_y_np = train_data.iloc[:,0].to_numpy().astype('float32')
train_x_np = train_data.iloc[:,1:].to_numpy().astype('float32')
## create RecordSet
formatted_train_data = linear.record_set(train_x_np, labels=train_y_np)
#
linear.fit(formatted_train_data)
### End of Solution 1

### Solution 2 Adaboost, 3 Neural Net MLPClassifier, 4 Nearest Neighbors, 5 Decision Tree
#estimator.fit({'train': os.path.join(train_and_test_path, 'train.csv'),
#               'test': os.path.join(train_and_test_path, 'test.csv')})
### End of Solution 2, 3, 4, 5


Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


2020-12-14 19:46:52 Starting - Starting the training job...ProfilerReport-1607975212: InProgress
...
2020-12-14 19:47:25 Starting - Launching requested ML instances......
2020-12-14 19:48:51 Starting - Preparing the instances for training.........
2020-12-14 19:50:19 Downloading - Downloading input data
2020-12-14 19:50:19 Training - Training image download completed. Training in progress..Docker entrypoint called with argument(s): train
Running default environment configuration script
[12/14/2020 19:50:22 INFO 140299200522048] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-input.json: {u'loss_insensitivity': u'0.01', u'epochs': u'15', u'feature_dim': u'auto', u'init_bias': u'0.0', u'lr_scheduler_factor': u'auto', u'num_calibration_samples': u'10000000', u'accuracy_top_k': u'3', u'_num_kv_servers': u'auto', u'use_bias': u'true', u'num_point_for_scaler': u'10000', u'_log_level': u'info', u'quantile': u'0.5', u'bias_lr_mult': u'auto

## EXERCISE: Deploy the trained model

After training, deploy your model to create a `predictor`. If you're using a PyTorch model, you'll need to create a trained `PyTorchModel` that accepts the trained `<model>.model_data` as an input parameter and points to the provided `source_pytorch/predict.py` file as an entry point. 

To deploy a trained model, you'll use `<model>.deploy`, which takes in two arguments:
* **initial_instance_count**: The number of deployed instances (1).
* **instance_type**: The type of SageMaker instance for deployment.

Note: If you run into an instance error, it may be because you chose the wrong training or deployment instance_type. It may help to refer to your previous exercise code to see which types of instances we used.

In [40]:
%%time

# uncomment, if needed
# from sagemaker.pytorch import PyTorchModel


# deploy your model to create a predictor

### Solution 1: LinearLearner
predictor = linear.deploy(initial_instance_count=1, instance_type='ml.t2.medium')
### End of Solution 1

### Solution 2 Adaboost, 3 Neural Net MLPClassifier, 4 Nearest Neighbors, 5 Decision Tree
#predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')
### End of Solution 2, 3, 4, 5

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


-------------------!CPU times: user 331 ms, sys: 32.6 ms, total: 363 ms
Wall time: 9min 32s


---
# Evaluating Your Model

Once your model is deployed, you can see how it performs when applied to our test data.

The provided cell below, reads in the test data, assuming it is stored locally in `data_dir` and named `test.csv`. The labels and features are extracted from the `.csv` file.

In [41]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import os

# read in test data, assuming it is stored locally
test_data = pd.read_csv(os.path.join(data_dir, "test.csv"), header=None, names=None)

# labels are in the first column
test_y = test_data.iloc[:,0]
test_x = test_data.iloc[:,1:]

## EXERCISE: Determine the accuracy of your model

Use your deployed `predictor` to generate predicted, class labels for the test data. Compare those to the *true* labels, `test_y`, and calculate the accuracy as a value between 0 and 1.0 that indicates the fraction of test data that your model classified correctly. You may use [sklearn.metrics](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics) for this calculation.

**To pass this project, your model should get at least 90% test accuracy.**

In [42]:
# First: generate predicted, class labels

### Solution 1 LinearLearner, 2 Adaboost, 3 Neural Net MLPClassifier, 4 Nearest Neighbors, 5 Decision Tree
test_y_preds = predictor.predict(test_x.to_numpy().astype('float32'))
### End Solution 1, 2, 3, 4, 5

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# test that your model generates the correct number of labels
assert len(test_y_preds)==len(test_y), 'Unexpected number of predictions.'
print('Test passed!')

Test passed!


In [43]:
# Second: calculate the test accuracy
from sklearn import metrics
from sklearn.metrics import accuracy_score


### Solution 1: LinearLearner: 0.36
predictions = [pred.label['predicted_label'].float32_tensor.values[0] for pred in test_y_preds]
tests = test_y.values
preds = np.array(list(map(int, predictions)))
print(accuracy_score(tests, preds))
### End Solution 1

### Solution 2 Adaboost: 0.44, 3 Neural Net MLPClassifier 0.44, 4 Nearest Neighbors 0.36, 5 Decision Tree
#tests = test_y.values
#preds = test_y_preds
#print(accuracy_score(tests, preds))
### End Solution 2, 3, 4, 5

tp = np.logical_and(tests, preds).sum()
fp = np.logical_and(1-tests, preds).sum()
tn = np.logical_and(1-tests, 1-preds).sum()
fn = np.logical_and(tests, 1-preds).sum()
print('{:<16} {}'.format('true positives', tp))
print('{:<16} {}'.format('false positives', fp))
print('{:<16} {}'.format('true negatives', tn))
print('{:<16} {}'.format('false negatives', fn))

print('{:<16} {:.2f}'.format('recall', tp / (tp + fn)))
print('{:<16} {:.2f}'.format('precision', tp / (tp + fp)))
print('{:<16} {:.2f}'.format('accuracy', (tp + tn) / (tp + fp + tn + fn)))



# print out the array of predicted and true labels, if you want
print('\nPredicted class labels / True class labels: ')
print(preds)
print(tests)
print(f'accuracy: {accuracy_score(tests, preds, normalize=False)}/{len(tests)} = {accuracy_score(tests, preds, normalize=False)/len(tests)}')

0.96
true positives   15
false positives  1
true negatives   9
false negatives  0
recall           1.00
precision        0.94
accuracy         0.96

Predicted class labels / True class labels: 
[1 1 1 1 1 1 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 1 1 0 1]
[1 1 1 1 1 1 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 1 1 0 0]
accuracy: 24/25 = 0.96


### Question 1: How many false positives and false negatives did your model produce, if any? And why do you think this is?

** Answer**: 
The model that I finally selected, had 1 false positive and 0 false negatives. 

From my point of view this is quite good a result. 

First, for a plagiarism detection program, it is good to have not many false negatives - that is, student answers that were wrongly classified as plagiarism. The damage of such cases could be potentially greater than having less plagiarism cases being undetected (false positives). In our case, there was not even one false negative case - recall was 100%. 

Second, there was only one false positive case - with precision being 94%. This is really not too bad for a plagiarism detection program. 

On the other hand, data was quite selective - it was especially produced to be used for a plagiarism detection model. And then data set was really small, with only 95 cases, both training and testing including. The model can be expected to have much lower accuracy with real life examples. 


### Question 2: How did you decide on the type of model to use? 

** Answer**:

There are two drawbacks of my decision on which model to select. 
First, I did not visualise the data space of positive vs. negative cases of plagiarism. This left me without any hint for selecting one of the types of sklearn models as described on the sklearn website, https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html.

Second, I failed to check my test data preparation step as soon as the first test runs of my models revealed no higher accurary than 50%. I continued with many models, and different feature selections (see the Second Notebook for more details), in order to find a better solution - which I apparently did not arrive at. 

When I finally spot the error and corrected it, I decided to stay with the final (third) approach towards the feature selection, having only one of the ngrams selected. It would have been interesting to have seen how the different models that I tried had done their job on the first and second approach, having 3 and 2 features selected respectively.

Hence the only approach I really follow was a trial and error approach. I wanted to have a LinearLearner in my models, and from the sklearn model I just selected some of them simply by chance. 

There were only few differences between the models when I tested with the defective data set in the beginning. I give a short overview on what I achieved. 

First Approach: accuracy with 3 features (c_1, c_6, lcs)
36% LinearLearner
36% Adaboost
36%-44% Neural Net MLPClassifier
32% Nearest Neigbors
N/A Decision Tree

Second Approach: accuracy with 2 features (c_1, c_6)
36% LinearLearner
44% Adaboost
36%-40% Neural Net MLPClassifier
36% Nearest Neighbors
40% Decision Tree

Third Approach: accuracy with 1 feature (c_3)
44% LinearLearner
52% Adaboost
44% Neural Net MLPClassifier
44% Nearest Neighbors
52% Decision Tree

Without having the wish to give these results too much weight - as the Class Labels of the Test Set were competely crap (I have selected the Labels from the Training Set instead ;-), it seems though that the 1-feature approach did reveal the best results. 

As already discussed, when I noticed that I had wrong Class Labels in the testing set, and after correcting it, I decided to stay with the third, 1-feature approach. So I cannot give any observations on how the model behaved in the first two approaches. The results were nonetheless very good so that I had no further reason to investigate into other feature selections. 

Here are the final results.

Third Approach: accuracy with 1 feature (c_3) - now with the CORRECT testing set!
96% LinearLearner
84% Adaboost
92% Neural Net MLPClassifier
92% Nearest Neighbors
84% Decision Tree

For the LinearLearner, I had also a more simple model in my list, which performed slightly less well with an accuracy of 92%. 
I finally selected the only model which did arrive at a very high accuracy and a good result with regards to false positives and false negatives. This is a LinearLearner with the 'precision_at_target_recall' feature, a target_precision at 0.9, and  balanced 'positive_example_weight_mult' (see above in this Notebook). 

A final note on PyTorch Models:
I only used LinearLearner and sklearn models. This was mainly due to the many difficulties I had arriving at a good result at all, so I spend my time with different feature selections. This finally led me to drop the initial plan to experiment a bit with some self-made PyTorch models, as well - which would have cost much more time in addition. 

----
## EXERCISE: Clean up Resources

After you're done evaluating your model, **delete your model endpoint**. You can do this with a call to `.delete_endpoint()`. You need to show, in this notebook, that the endpoint was deleted. Any other resources, you may delete from the AWS console, and you will find more instructions on cleaning up all your resources, below.

In [44]:
# uncomment and fill in the line below!
predictor.delete_endpoint()


### Deleting S3 bucket

When you are *completely* done with training and testing models, you can also delete your entire S3 bucket. If you do this before you are done training your model, you'll have to recreate your S3 bucket and upload your training data again.

In [45]:
# deleting bucket, uncomment lines below

bucket_to_delete = boto3.resource('s3').Bucket(bucket)
bucket_to_delete.objects.all().delete()

[{'ResponseMetadata': {'RequestId': '706232FF436BA679',
   'HostId': 'FMybhxQ5ZOeZZUmTfDAlQsd9E69TAPqgRT66FMz3myoK+/YYdICWRUcQ96fLvOnWJHBmHxKP5s0=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'FMybhxQ5ZOeZZUmTfDAlQsd9E69TAPqgRT66FMz3myoK+/YYdICWRUcQ96fLvOnWJHBmHxKP5s0=',
    'x-amz-request-id': '706232FF436BA679',
    'date': 'Mon, 14 Dec 2020 21:59:24 GMT',
    'connection': 'close',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'plagiarism_output/linear-learner-2020-12-13-23-45-21-363/rule-output/ProfilerReport-1607903121/profiler-output/profiler-reports/BatchSize.json'},
   {'Key': 'plagiarism_output/sagemaker-scikit-learn-2020-12-14-19-28-05-217/profiler-output/system/training_job_end.ts'},
   {'Key': 'plagiarism_output/sagemaker-scikit-learn-2020-12-14-19-04-25-704/rule-output/ProfilerReport-1607972665/profiler-output/profiler-reports/OverallFrameworkMetrics.json'}

### Deleting all your models and instances

When you are _completely_ done with this project and do **not** ever want to revisit this notebook, you can choose to delete all of your SageMaker notebook instances and models by following [these instructions](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-cleanup.html). Before you delete this notebook instance, I recommend at least downloading a copy and saving it, locally.

---
## Further Directions

There are many ways to improve or add on to this project to expand your learning or make this more of a unique project for you. A few ideas are listed below:
* Train a classifier to predict the *category* (1-3) of plagiarism and not just plagiarized (1) or not (0).
* Utilize a different and larger dataset to see if this model can be extended to other types of plagiarism.
* Use language or character-level analysis to find different (and more) similarity features.
* Write a complete pipeline function that accepts a source text and submitted text file, and classifies the submitted text as plagiarized or not.
* Use API Gateway and a lambda function to deploy your model to a web application.

These are all just options for extending your work. If you've completed all the exercises in this notebook, you've completed a real-world application, and can proceed to submit your project. Great job!